### 0. Prior Setting

In [ ]:
import numpy as np
import random
import math
from tqdm import tqdm
from environment import Communicator
import threading

In [ ]:
debug_mode = False
STEP = 150

In [ ]:
FPS = 30
ARRIVAL_MAX = 1.0
test_multi = True

In [ ]:
def send(omnet, time):
    omnet.get_omnet_message()
    omnet.send_omnet_message("action")
    omnet.get_omnet_message()
    omnet.send_omnet_message(str(time))
    return True

In [ ]:
def receive(omnet):
    omnet.get_omnet_message()
    omnet.send_omnet_message("reward")
    path_cost = omnet.get_omnet_message()
    omnet.send_omnet_message("ACK")
    return float(path_cost)

In [ ]:
def setting_omnet(pipe_num):
    if pipe_num == 1:
        omnet = Communicator("\\\\.\\pipe\\frame_drop_rl", 200000, debug_mode=debug_mode)
    else :
        omnet = Communicator("\\\\.\\pipe\\frame_drop_rl_"+str(pipe_num), 200000, debug_mode=debug_mode)
    return omnet

In [ ]:
def testing(pipenum, V, hard=False):
    omnet = setting_omnet(pipenum)
    
    for epi in tqdm(range(STEP)):
        if hard: action = 0
        else: action = random.randint(0, FPS)
        for idx in range(FPS+1):
            if idx == action: send(omnet, (action+1)/FPS)
            elif idx > action: send(omnet, 1/FPS)

        path_cost = receive(omnet)
        arrival_rate = ARRIVAL_MAX if path_cost == 0 else min(ARRIVAL_MAX, V / path_cost)
        arrival_frame_num = math.floor(arrival_rate*FPS)
        
        print(f"action: {action}, path_cost: {path_cost}, max arrival rate with V: {arrival_rate}, max arrival frame num: {arrival_frame_num}")


    omnet.get_omnet_message()
    omnet.send_omnet_message("finish")

### 1. Checking YOLO env [Config Wireless-Node2-1]

cost가 1000~10000단위로 나오기 때문에 V도 그에 맞추어서 조절하면 잘 진행되는 것을 확인

In [ ]:
V = 1.0

In [ ]:
if not test_multi:
    testing(pipenum=1, V=V, hard=True)

### 2. Checking abilene env [Config Wireless11-Node1Node3node5-1]


In [ ]:
V= 100000

In [ ]:
if test_multi: 
    function_list = [testing, testing, testing]
    pipenum_list = [i+1 for i in range(3)]
    
    print(pipenum_list)

In [ ]:
if test_multi: 
    threads = []
    
    for func, pipe in zip(function_list, pipenum_list):
        thread = threading.Thread(target=func, args=(pipe, V))
        threads.append(thread)
        thread.start()

In [ ]:
if test_multi: 
    for thread in threads:
        thread.join()

    print("All functions have completed execution.")